In [ ]:
import os
import sys
import torch
import numpy as np

# Đảm bảo import đúng các module nội bộ
sys.path.append(os.path.abspath(os.path.dirname(__file__)))
from net.classifier import Classifier
from generator_cvae.utils.common import get_affective_features

# Hàm load model hybrid

def load_hybrid_model(
    model_dir,
    in_channels=3,
    in_features=48,  # Số lượng affective features
    num_classes=4,
    joints=16,
    temporal_kernel_size=75,
    device='cpu'
):
    graph_dict = {'strategy': 'spatial', 'layout': 'ntu-rgb+d', 'num_node': joints}
    model = Classifier(
        in_channels=in_channels,
        in_features=in_features,
        num_classes=num_classes,
        graph_args=graph_dict,
        temporal_kernel_size=temporal_kernel_size
    )
    model = model.to(device)
    # Tìm file trọng số tốt nhất
    weight_files = [f for f in os.listdir(model_dir) if f.endswith('.pth.tar')]
    assert weight_files, "Không tìm thấy file trọng số"
    best_weight = sorted(weight_files, key=lambda x: float(x.split('_acc')[1].split('_')[0]), reverse=True)[0]
    weight_path = os.path.join(model_dir, best_weight)
    state_dict = torch.load(weight_path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()
    return model

# Hàm dự đoán cảm xúc từ 1 mảng khung xương

def predict_emotion_from_gait(model, gait, device='cpu'):
    """
    gait: numpy array shape (T, joints*coords) hoặc (T, joints, coords)
    """
    in_channels = 3
    joints = 16
    # Đảm bảo shape (T, joints, coords)
    if gait.ndim == 2 and gait.shape[1] == joints * in_channels:
        gait = gait.reshape(gait.shape[0], joints, in_channels)
    elif gait.ndim == 3 and gait.shape[1] == joints and gait.shape[2] == in_channels:
        pass
    else:
        raise ValueError("Shape gait phải là (T, joints*coords) hoặc (T, joints, coords)")
    # Chuẩn hóa shape cho get_affective_features: (1, T, joints, coords)
    gait_input = gait[np.newaxis, ...]  # (1, T, joints, coords)
    aff_all = get_affective_features(gait_input)  # (1, T, 175)
    aff = aff_all[0, :, :48]  # Lấy 48 affective features đầu tiên cho mỗi frame
    aff_vec = aff.mean(axis=0)  # (48,)
    # Chuẩn hóa shape cho model hybrid
    T = gait.shape[0]
    gait_for_model = gait.transpose(2,0,1)  # (coords, T, joints)
    gait_for_model = gait_for_model[np.newaxis, ...]  # (1, coords, T, joints)
    gait_for_model = gait_for_model[..., np.newaxis]  # (1, coords, T, joints, 1)
    gait_tensor = torch.from_numpy(gait_for_model).float().to(device)
    aff_tensor = torch.from_numpy(aff_vec).float().unsqueeze(0).to(device)  # (1, 48)
    with torch.no_grad():
        output = model(aff_tensor, gait_tensor)
        pred_label = torch.argmax(output, dim=1).item()
    return pred_label

if __name__ == '__main__':
    import argparse
    parser = argparse.ArgumentParser(description='Phan loai cam xuc tu khung xuong voi model hybrid')
    parser.add_argument('--model-dir', type=str, required=True, help='Thư mục chứa trọng số model hybrid (.pth.tar)')
    parser.add_argument('--input-npy', type=str, required=True, help='File .npy chứa mảng khung xương shape (T, joints*coords) hoặc (T, joints, coords)')
    parser.add_argument('--device', type=str, default='cpu', help='cpu hoặc cuda:0')
    args = parser.parse_args()

    # Load dữ liệu khung xương
    gait = np.load(args.input_npy)
    # Load model
    model = load_hybrid_model(args.model_dir, device=args.device)
    # Dự đoán
    label = predict_emotion_from_gait(model, gait, device=args.device)
    print('Emotion label:', label)


In [30]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'D:\PBL4\STEP\VID_RGB_000.csv', header = None)
df = np.array(df)
df.shape
# gait = df
time_step = 1.0 / 30.0
aff = []
# aff = compute_features(df,time_step)
aff.append(compute_features(df, time_step))
norma = []
normalize_features(aff,norma)

In [31]:
norma

[[0.00012375484448748497,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [35]:
aff.shape

AttributeError: 'list' object has no attribute 'shape'

In [34]:
import pandas as pd
import os

h5_path = r"D:\PBL4\STEP\Data EWalk\affectiveFeatures_3D.h5"
output_folder = r"D:\PBL4\STEP\output_folder"

# Tạo thư mục output nếu chưa có
os.makedirs(output_folder, exist_ok=True)

# Mở file HDF5
with pd.HDFStore(h5_path, mode='r') as store:
    for key in store.keys():
        # key có dạng '/VID_RGB_000', ta loại bỏ dấu '/' đầu tiên
        filename = key.strip("/") + ".csv"
        output_path = os.path.join(output_folder, filename)
        
        # Đọc từng dataset và lưu ra CSV
        df = store[key]
        df.to_csv(output_path, index=False, header=False)
        print(f"Đã lưu {output_path}")


In [10]:
base_path = r'D:\PBL4\STEP\Data EWalk'
# D:\STEP\Data EWalk\Data_2D2.h5
# Get affective features
f_type = ''
positions = h5py.File(os.path.join(base_path, 'Data_2D2.h5'.format(f_type)), 'r')
keys = positions.keys()
time_step = 1.0 / 30.0

print('Computing Features ... ', end='')
features = []
for key in keys:
	frames = positions[key]
	print(frames)
	feature = [key]
	if frames.ndim == 2:
		feature += compute_features(frames, time_step)
	else:
		feature += frames
	features.append(feature)


Computing Features ... <HDF5 dataset "VID_RGB_000.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_001.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_002.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_003.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_004.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_005.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_006.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_007.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_008.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_009.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_010.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_011.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_012.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_013.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_014.csv": shape (30, 48), type "<f8">
<HDF5 dataset "VID_RGB_015.csv": shape (30, 48), type "<f8">
<

In [12]:
features

[['VID_RGB_000.csv',
  5.555411514597205,
  150.29606073583335,
  14.504530873719798,
  15.19940839044684,
  175.98456335445863,
  173.47661763605655,
  4.131083188987161,
  4.076200127644705,
  10.577519525414258,
  10.39607852890259,
  31.83865221396097,
  9.364001273045469,
  6.73744496230306,
  20.87865967636174,
  18.03044223061495,
  12.292072417710957,
  9.719072038182599,
  181.07076534644187,
  1051.2188894549868,
  854.201229840763,
  251.2249719157509,
  238.68390692189655,
  9539.605376889695,
  63182.20059567509,
  47211.63413577625,
  13296.16926464236,
  11454.709957997797,
  4.875,
  0.1625],
 ['VID_RGB_001.csv',
  5.849648121606042,
  152.24209768499986,
  13.840435561455651,
  13.917466753544538,
  176.50533743110023,
  173.05945227833976,
  4.18510002698066,
  4.063782978282089,
  10.65975804826962,
  10.435510495945005,
  32.630116658066946,
  9.471943261024455,
  7.519857110746345,
  7.166002539967334,
  15.266418861438137,
  12.746853332949875,
  10.95871937006010

In [ ]:
print('done.\nNormalizing Features ... ', end='')
normalized_features = []
normalize_features(features, normalized_features)
print('done.\nSaving features ... ', end='')
with h5py.File(r'D:\STEP\Data EWalk\affectiveFeatures_2D.h5'.format(f_type), 'w') as aff:
	for feature in normalized_features:
		aff.create_dataset(feature[0], data=feature[1:])
print('done.')

# Get labels
# labels = h5py.File(os.path.join(base_path, 'data', 'labels{}.h5'.format(f_type)), 'r')

# Cross validate
# cross_validate(normalized_features, labels)
